### Melon 100 챠트 스크래핑
1. 100곡의 노래의 제목과 ID 추출 (정규표현식)
2. 노래의 ID로 상세페이지를 100번 요청해서 상세정보를 추출
3. Json file에 저장
4. json file을 DataFrame 객체로 생성해서 표데이터 만들기
5. 표데이터를 DB에 저장하기

In [18]:
import requests
from bs4 import BeautifulSoup
import re

url = 'https://www.melon.com/chart/index.htm'
# 요청헤더 
req_header = {
    'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36'
}
res = requests.get(url, headers=req_header)
print(res.status_code)

if res.ok:
    html = res.text
    soup = BeautifulSoup(html, 'html.parser')
    # <div id="tb_list"> 
    print(len(soup.select('div#tb_list'))) #1
    # <div id="tb_list">  하위의 tr 선택
    print(len(soup.select('div#tb_list tr'))) #101
    # <div id="tb_list">  하위의 tr 하위의 <div class="wrap_song_info">
    print(len(soup.select('div#tb_list tr div.wrap_song_info'))) #200
    # <div id="tb_list">  하위의 tr 하위의 <div class="wrap_song_info"> 하위의 <a href="playSong">
    print(len(soup.select("div#tb_list tr div.wrap_song_info a[href*='playSong']"))) #100
    
    song_atag = soup.select("div#tb_list tr div.wrap_song_info a[href*='playSong']")
    # 100개의 song정보를 저장할 리스트 선언
    song_list = []
    for song in song_atag:
        # 1개의 song 정보를 저장한 딕셔너리 선언
        song_dict = {}
        title = song.text
        href_value = song['href']
        matched = re.search(r'(\d+)\);', href_value)
        #print(matched.group(0), matched.group(1))
        song_id = matched.group(1)
        
        print(title, song_id)

200
1
101
200
100
30287019); 30287019
롤린 (Rollin') 30287019
33372781); 33372781
라일락 33372781
33359317); 33359317
Peaches (Feat. Daniel Caesar & Giveon) 33359317
33239419); 33239419
Celebrity 33239419
32860397); 32860397
운전만해 (We Ride) 32860397
1944399); 1944399
Timeless 1944399
1854856); 1854856
라라라 1854856
1500196); 1500196
내사람 1500196
33372783); 33372783
Coin 33372783
3414749); 3414749
내 손을 잡아 3414749
32872978); 32872978
Dynamite 32872978
33346446); 33346446
On The Ground 33346446
33331004); 33331004
LOVE DAY (2021) (바른연애 길잡이 X 양요섭, 정은지) 33331004
33167063); 33167063
밝게 빛나는 별이 되어 비춰줄게 33167063
33061995); 33061995
밤하늘의 별을(2020) 33061995
33405149); 33405149
그냥 안아달란 말야 33405149
711626); 711626
살다가 711626
33013877); 33013877
잠이 오질 않네요 33013877
32183386); 32183386
Blueming 32183386
32578498); 32578498
에잇(Prod.&Feat. SUGA of BTS) 32578498
32794652); 32794652
취기를 빌려 (취향저격 그녀 X 산들) 32794652
33077590); 33077590
VVS (Feat. JUSTHIS) (Prod. GroovyRoom) 33077590
33372788); 33372788
어푸 (Ah puh) 333